In [1]:
# libraries
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
# our modules
from preprocessing import *

In [2]:
torneo = pd.read_csv('data/2020.csv', 
                            encoding='utf-8-sig', 
                            dtype=DATA_TYPES,
                            parse_dates=['Date', 'WBD', 'LBD'])
torneo['Round'].unique()

array(['1st Round', '2nd Round', 'Quarterfinals', 'Semifinals',
       'The Final'], dtype=object)

In [16]:
features_to_drop = ['Best of', 'Court','WElo', 'WSurfElo', 'WHand', 'WBHand',
                    'LElo', 'LSurfElo', 'LHand', 'LBHand','Surface']
X, Y = preprocess_data(2011, 2019, features_to_drop)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort=False,
                                                        random_state=None,
                                                        splitter='best'),
 

In [30]:
# Build the model
bdt = BaggingClassifier(DecisionTreeClassifier(criterion='entropy'), 
                        bootstrap=True,
                        n_estimators=200,
                        max_samples=0.8,
                        n_jobs=-1)    
bdt.fit(X, Y)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort=False,
                                                        random_state=None,
                                                        splitter='best'),
 

In [34]:
def simulation(model,
               atp = 1, 
               features_to_drop=[], 
               missing_values="drop", 
               drop_first=False):
    
    tournament = pd.read_csv("data/" + str(2020) + ".csv", encoding='utf-8-sig', dtype=DATA_TYPES)
    tournament = tournament[tournament['ATP'] == atp]
    # TODO filtrare per atp e round(primo)
    
    tournament_to_test = unify_data(tournament, features_to_drop, 'drop', drop_first)
    
    prediction = model.predict(tournament_to_test[tournament_to_test['Round'] == 0])
    accuracy = round(accuracy_score(y_true=np.ones(len(prediction), dtype=int), y_pred=prediction), 3)
    print("Accuracy: ", accuracy)
    
    firstR = tournament[tournament['Round'] == '1st Round']
    i = 0
    for j, match in firstR.iterrows():
        print('P1: ' + match['Winner'], 'P2: '+ match['Loser'], 'Wins:', prediction[i])
        i+=1

    #print(prediction)
    
    return tournament_to_test, X, Y
    

In [35]:
T, X, Y = simulation(bdt, 1, features_to_drop)

Accuracy:  0.75
P1: Bublik A. P2: Mannarino A. Wins: 0
P1: Moutet C. P2: Sandgren T. Wins: 1
P1: Verdasco F. P2: Andujar P. Wins: 1
P1: Bedene A. P2: Ymer M. Wins: 1
P1: Chardy J. P2: Barrere G. Wins: 1
P1: Kecmanovic M. P2: Thompson J. Wins: 1
P1: Krajinovic F. P2: Edmund K. Wins: 1
P1: Herbert P.H. P2: Cecchinato M. Wins: 0
P1: Kukushkin M. P2: Jaziri M. Wins: 0
P1: Fucsovics M. P2: Tiafoe F. Wins: 1
P1: Djere L. P2: Sonego L. Wins: 1
P1: Ilkel C. P2: Berankis R. Wins: 1


In [45]:
print(T.iloc[0],T.iloc[4], T.iloc[7], T.iloc[8])

Series            0
Round             0
WRank            55
LRank            43
WPts            919
LPts           1111
MaxW           2.25
MaxL            1.8
AvgW           2.11
AvgL           1.72
WEloCalc       1500
LEloCalc       1500
GreaterRank       0
Name: 0, dtype: object Series            0
Round             0
WRank            54
LRank            83
WPts            920
LPts            636
MaxW           1.57
MaxL           2.65
AvgW           1.53
AvgL           2.47
WEloCalc       1500
LEloCalc       1500
GreaterRank       1
Name: 4, dtype: object Series            0
Round             0
WRank            39
LRank            51
WPts           1151
LPts            990
MaxW            3.2
MaxL           1.43
AvgW           3.03
AvgL           1.37
WEloCalc       1500
LEloCalc       1516
GreaterRank       1
Name: 12, dtype: object Series            0
Round             0
WRank            70
LRank            47
WPts            790
LPts           1050
MaxW            2.1
MaxL      

Notare che il modello sbaglia la predizione quando anche le scommesse(Max e Avg) sbagliano(era più quotata la vittoria del giocatore che di fatto ha perso)
TODO: 
    - fare un decisore stupido basato sulle scommesse
    - provare a tenere B365, nel caso droppare Avg e Max